In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
from langchain_community.document_loaders import DirectoryLoader
from dotenv import load_dotenv
import os

# Загрузка необходимого ресурса для NLTK
nltk.download('punkt')

# Загрузка переменных окружения
load_dotenv()

# Инициализация загрузчика
loader = DirectoryLoader('data', glob="**/*.txt")

# Загрузка документов
documents = loader.load()


[nltk_data] Downloading package punkt to /Users/master/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
print(documents)
print(documents[0].page_content)

[Document(page_content='Q: Какими бывают кукусики?\n\nA: Кукусики бывают фельдепёрстовые.\n\nQ: Что делает нашу пиццу уникальной? A: Наша пицца готовится по секретному семейному рецепту теста, вручную раскатывается и украшается свежими, местными ингредиентами. Мы используем традиционную дровяную печь, чтобы придать пицце характерный дымный вкус и хрустящую корочку.\n\nQ: Предлагаем ли мы варианты пиццы без глютена? A: Да, у нас есть вкусная безглютеновая корочка для гостей с диетическими ограничениями или предпочтениями.\n\nQ: Могут ли клиенты создать свою собственную пиццу? A: Конечно! Клиенты могут выбрать из множества свежих топпингов, чтобы создать свою уникальную пиццу-шедевр.\n\nQ: Какие наши самые популярные пиццы? A: Наши самые популярные пиццы — это Классическая Маргарита, Пепперони Суприм и Гурман Вегетарианская. Каждая из них предлагает уникальное сочетание вкусов, которое удовлетворяет разнообразные предпочтения.\n\nQ: Предлагаем ли мы веганские варианты пиццы? A: Да, у нас

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=250,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

In [4]:
chunks = text_splitter.split_documents(documents)
chunks

Created a chunk of size 274, which is longer than the specified 250


[Document(page_content='Q: Какими бывают кукусики?\n\nA: Кукусики бывают фельдепёрстовые.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Что делает нашу пиццу уникальной? A: Наша пицца готовится по секретному семейному рецепту теста, вручную раскатывается и украшается свежими, местными ингредиентами. Мы используем традиционную дровяную печь, чтобы придать пицце характерный дымный вкус и хрустящую корочку.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Предлагаем ли мы варианты пиццы без глютена? A: Да, у нас есть вкусная безглютеновая корочка для гостей с диетическими ограничениями или предпочтениями.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Могут ли клиенты создать свою собственную пиццу? A: Конечно! Клиенты могут выбрать из множества свежих топпингов, чтобы создать свою уникальную пиццу-шедевр.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Какие наши самые популярные пиццы? A: Наши самые популя

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

index = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [6]:
retriever = index.as_retriever()

In [7]:
retriever.get_relevant_documents("Какими бывают кукусики")

[Document(page_content='Q: Какими бывают кукусики?\n\nA: Кукусики бывают фельдепёрстовые.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Какими бывают рештальдовые кукусики? A: Рештальдовые кукусики бывают исключительно калдобровсчатыми.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Какие наши самые популярные пиццы? A: Наши самые популярные пиццы — это Классическая Маргарита, Пепперони Суприм и Гурман Вегетарианская. Каждая из них предлагает уникальное сочетание вкусов, которое удовлетворяет разнообразные предпочтения.', metadata={'source': 'data/text.txt'}),
 Document(page_content='Q: Что делает нашу пиццу уникальной? A: Наша пицца готовится по секретному семейному рецепту теста, вручную раскатывается и украшается свежими, местными ингредиентами. Мы используем традиционную дровяную печь, чтобы придать пицце характерный дымный вкус и хрустящую корочку.', metadata={'source': 'data/text.txt'})]

In [8]:
from operator import itemgetter

template = """
Ответь на вопрос используя следующий контекст:
{context}

Ответь на этот вопрос:
Вопрос: {question}
"""

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [10]:
rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
rag_chain.invoke({"question": "Какими бывают кукусики"})

'Ответ: Кукусики бывают фельдепёрстовые и рештальдовые, причем рештальдовые кукусики бывают исключительно калдобровсчатыми.'

In [13]:
rag_chain.invoke({"question": "Долго готовится пицца?"})


'Ответ: В среднем, от момента заказа до готовности пиццы проходит около 15-20 минут.'